## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

# Summarization

Text summarization involves condensing lengthy textual content into a brief format while retaining its essential information and significance. The primary aim is to extract key details from a text document and present them concisely and comprehensibly. It plays a crucial role across various domains, including healthcare, aiding in efficient communication and decision-making.

- **Model**: `en.summarize.clinical_jsl_augmented.pipeline`
- **Model Description**: This pretrained pipeline is built on the top of summarizer_clinical_jsl_augmented model, which is is a modified version of Flan-T5 (LLM) based summarization model that is at first finetuned with natural instructions and then finetuned with clinical notes, encounters, critical care notes, discharge notes, reports, curated  by John Snow Labs. This model is further optimized by augmenting the training methodology, and dataset. It can generate summaries from clinical notes up to 512 tokens given the input text (max 1024 tokens).


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "en-summarize-clinical-jsl-augmented-pipeline"

content_type = "application/json"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"

### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Perform real-time inference

In [6]:
import json
import pandas as pd
import os
import boto3


# Set display options
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None) 


def process_data_and_invoke_realtime_endpoint(data_dicts):
    for data_dict in data_dicts:
        json_input_data = json.dumps(data_dict)    
        i = 1
        input_file_name = f'inputs/real-time/input{i}.json'
        output_file_name = f'outputs/real-time/out{i}.out'
        
        while os.path.exists(input_file_name) or os.path.exists(output_file_name):
            i += 1
            input_file_name = f'inputs/real-time/input{i}.json'
            output_file_name = f'outputs/real-time/out{i}.out'

        os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
        os.makedirs(os.path.dirname(output_file_name), exist_ok=True)
        
        with open(input_file_name, 'w') as f:
            f.write(json_input_data)
        
        s3_client.put_object(Bucket=s3_bucket, Key=f"validation-input-json/{os.path.basename(input_file_name)}", Body=bytes(json_input_data.encode('UTF-8')))
        
        response = sm_runtime.invoke_endpoint(
            EndpointName=model_name,
            ContentType=content_type,
            Accept="application/json",
            Body=json_input_data,
        )

        # Process response
        response_data = json.loads(response["Body"].read().decode("utf-8"))

        df = pd.DataFrame(response_data, columns=['summary'])
        display(df)

        # Save response data to file
        with open(output_file_name, 'w') as f_out:
            json.dump(response_data, f_out, indent=4)


  **Input format**:
  
  
{
  "text": "Input text that you want to summarize."
}


In [7]:
# Example usage:
data_dicts = [
{"text" : ['''Medical Specialty: Allergy / Immunology, Sample Name: Allergic Rhinitis
Description: A 23-year-old white female presents with complaint of allergies. (Medical Transcription Sample Report)


SUBJECTIVE: This 23-year-old white female presents with complaint of allergies. She used to have allergies when she lived in Seattle but she thinks they are worse here. In the past, she has tried Claritin, and Zyrtec. Both worked for short time but then seemed to lose effectiveness. She has used Allegra also. She used that last summer and she began using it again two weeks ago. It does not appear to be working very well. She has used over-the-counter sprays but no prescription nasal sprays. She does have asthma but doest not require daily medication for this and does not think it is flaring up.


MEDICATIONS: Her only medication currently is Ortho Tri-Cyclen and the Allegra.


ALLERGIES: She has no known medicine allergies.


OBJECTIVE: Vitals: Weight was 130 pounds and blood pressure 124/78. HEENT: Her throat was mildly erythematous without exudate. Nasal mucosa was erythematous and swollen. Only clear drainage was seen. TMs were clear. Neck: Supple without adenopathy. Lungs: Clear.


ASSESSMENT: Allergic rhinitis.


PLAN:


1- She will try Zyrtec instead of Allegra again. Another option will be to use loratadine. She does not think she has prescription coverage so that might be cheaper.


2- Samples of Nasonex two sprays in each nostril given for three weeks. A prescription was written as well.''',

'''Medical Specialty: Pediatrics - Neonatal, Sample Name: Chest Closure
Description: Delayed primary chest closure. Open chest status post modified stage 1 Norwood operation. The patient is a newborn with diagnosis of hypoplastic left heart syndrome who 48 hours prior to the current procedure has undergone a modified stage 1 Norwood operation. (Medical Transcription Sample Report)


PROCEDURE: Delayed primary chest closure.


INDICATIONS: The patient is a newborn with diagnosis of hypoplastic left heart syndrome who 48 hours prior to the current procedure has undergone a modified stage 1 Norwood operation. Given the magnitude of the operation and the size of the patient (2.5 kg), we have elected to leave the chest open to facilitate postoperative management. He is now taken back to the operative room for delayed primary chest closure.


PREOP DX: Open chest status post modified stage 1 Norwood operation.


POSTOP DX: Open chest status post modified stage 1 Norwood operation.


ANESTHESIA: General endotracheal.


COMPLICATIONS: None.


FINDINGS: No evidence of intramediastinal purulence or hematoma. He tolerated the procedure well.


DETAILS OF PROCEDURE: The patient was brought to the operating room and placed on the operating table in the supine position. Following general endotracheal anesthesia, the chest was prepped and draped in the usual sterile fashion. The previously placed AlloDerm membrane was removed. Mediastinal cultures were obtained, and the mediastinum was then profusely irrigated and suctioned. Both cavities were also irrigated and suctioned. The drains were flushed and repositioned. Approximately 30 cubic centimeters of blood were drawn slowly from the right atrial line. The sternum was then smeared with a vancomycin paste. The proximal aspect of the 5 mm RV-PA conduit was marked with a small titanium clip at its inferior most aspect and with an additional one on its rightward inferior side. The sternum was then closed with stainless steel wires followed by closure of subcutaneous tissues with interrupted monofilament stitches. The skin was closed with interrupted nylon sutures and a sterile dressing was placed. The peritoneal dialysis catheter, atrial and ventricular pacing wires were removed. The patient was transferred to the pediatric intensive unit shortly thereafter in very stable condition. I was the surgical attending present in the operating room and in charge of the surgical procedure throughout the entire length of the case.''']
}
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,summary
0,"A 23-year-old female with a history of allergies and asthma presents with worsening symptoms. She has tried Claritin and Zyrtec but they haven't worked well. She has tried Allegra and has tried over-the-counter sprays but no prescription nasal sprays. Her vital signs are stable, and she has mildly erythematous throat and nasal mucosa. She is diagnosed with allergic rhinitis and will try Zyrtec or loratadine. Samples of Nasonex were given for three weeks."
1,"A newborn with hypoplastic left heart syndrome underwent a modified stage 1 Norwood operation. The chest was left open to facilitate postoperative management, and the patient tolerated the procedure well. The sternum was closed with stainless steel wires, subcutaneous tissues with interrupted monofilament stitches, and the skin was closed with interrupted nylon sutures. The patient was transferred to the pediatric intensive unit in stable condition."


In [8]:
data_dicts = [
    {"text": '''Medical Specialty: Surgery, Sample Name: Achilles Tendon Repair
Description: Achilles tendon rupture, left lower extremity. Primary repair left Achilles tendon. The patient was stepping off a hilo at work when he felt a sudden pop in the posterior aspect of his left leg. The patient was placed in posterior splint and followed up at ABC orthopedics for further care. (Medical Transcription Sample Report)


PREOPERATIVE DIAGNOSIS: Achilles tendon rupture, left lower extremity.


POSTOPERATIVE DIAGNOSIS: Achilles tendon rupture, left lower extremity.


PROCEDURE PERFORMED: Primary repair left Achilles tendon.


ANESTHESIA: General.


COMPLICATIONS: None.


ESTIMATED BLOOD LOSS: Minimal.


TOTAL TOURNIQUET TIME: 40 minutes at 325 mmHg.


POSITION: Prone.


HISTORY OF PRESENT ILLNESS: The patient is a 26-year-old African-American male who states that he was stepping off a hilo at work when he felt a sudden pop in the posterior aspect of his left leg. The patient was placed in posterior splint and followed up at ABC orthopedics for further care.


PROCEDURE: After all potential complications, risks, as well as anticipated benefits of the above-named procedure were discussed at length with the patient, informed consent was obtained. The operative extremity was then confirmed with the patient, the operative surgeon, Department Of Anesthesia, and nursing staff. While in this hospital, the Department Of Anesthesia administered general anesthetic to the patient. The patient was then transferred to the operative table and placed in the prone position. All bony prominences were well padded at this time. A nonsterile tourniquet was placed on the left upper thigh of the patient, but not inflated at this time. Left lower extremity was sterilely prepped and draped in the usual sterile fashion. Once this was done, the left lower extremity was elevated and exsanguinated using an Esmarch and the tourniquet was inflated to 325 mmHg and kept up for a total of 40 minutes. After all bony and soft tissue land marks were identified, a 6 cm longitudinal incision was made paramedial to the Achilles tendon from its insertion proximal. Careful dissection was then taken down to the level of the peritenon. Once this was reached, full thickness flaps were performed medially and laterally. Next, retractor was placed. All neurovascular structures were protected. A longitudinal incision was then made in the peritenon and opened up exposing the tendon. There was noted to be complete rupture of the tendon approximately 4 cm proximal to the insertion point. The plantar tendon was noted to be intact. The tendon was debrided at this time of hematoma as well as frayed tendon. Wound was copiously irrigated and dried. Most of the ankle appeared that there was sufficient tendon links in order to do a primary repair. Next #0 PDS on a taper needle was selected and a Krackow stitch was then performed. Two sutures were then used and tied individually ________ from the tendon. The tendon came together very well and with a tight connection. Next, a #2-0 Vicryl suture was then used to close the peritenon over the Achilles tendon. The wound was once again copiously irrigated and dried. A #2-0 Vicryl sutures were then used to close the skin and subcutaneous fashion followed by #4-0 suture in the subcuticular closure on the skin. Steri-Strips were then placed over the wound and the sterile dressing was applied consisting of 4x4s, Kerlix roll, sterile Kerlix and a short length fiberglass cast in a plantar position. At this time, the Department of anesthesia reversed the anesthetic. The patient was transferred back to hospital gurney to the Postanesthesia Care Unit. The patient tolerated the procedure well. There were no complications.'''}
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,summary
0,A 26-year-old male underwent a primary repair of an Achilles tendon in his left lower extremity due to a sudden pop. The procedure was performed under general anesthesia without complications. The tendon was debrided and the wound was closed with sutures. The patient tolerated the procedure well and was transferred to the Postanesthesia Care Unit.


### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [9]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [10]:
validation_input_path = f"s3://{s3_bucket}/validation-input-json/"
validation_output_path = f"s3://{s3_bucket}/validation-output-json/"

In [11]:
input_file_name = 'inputs/batch/input.json'
output_file_name = 'outputs/batch/out.out'

os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    accept="application/json",
)
transformer.transform(validation_input_path, content_type=content_type)
transformer.wait()

In [ ]:
validation_file_name = "input.json"
input_json_data = {"text" : ['''Medical Specialty: Allergy / Immunology, Sample Name: Allergic Rhinitis
Description: A 23-year-old white female presents with complaint of allergies. (Medical Transcription Sample Report)


SUBJECTIVE: This 23-year-old white female presents with complaint of allergies. She used to have allergies when she lived in Seattle but she thinks they are worse here. In the past, she has tried Claritin, and Zyrtec. Both worked for short time but then seemed to lose effectiveness. She has used Allegra also. She used that last summer and she began using it again two weeks ago. It does not appear to be working very well. She has used over-the-counter sprays but no prescription nasal sprays. She does have asthma but doest not require daily medication for this and does not think it is flaring up.


MEDICATIONS: Her only medication currently is Ortho Tri-Cyclen and the Allegra.


ALLERGIES: She has no known medicine allergies.


OBJECTIVE: Vitals: Weight was 130 pounds and blood pressure 124/78. HEENT: Her throat was mildly erythematous without exudate. Nasal mucosa was erythematous and swollen. Only clear drainage was seen. TMs were clear. Neck: Supple without adenopathy. Lungs: Clear.


ASSESSMENT: Allergic rhinitis.


PLAN:


1- She will try Zyrtec instead of Allegra again. Another option will be to use loratadine. She does not think she has prescription coverage so that might be cheaper.


2- Samples of Nasonex two sprays in each nostril given for three weeks. A prescription was written as well.''',

'''Medical Specialty: Pediatrics - Neonatal, Sample Name: Chest Closure
Description: Delayed primary chest closure. Open chest status post modified stage 1 Norwood operation. The patient is a newborn with diagnosis of hypoplastic left heart syndrome who 48 hours prior to the current procedure has undergone a modified stage 1 Norwood operation. (Medical Transcription Sample Report)


PROCEDURE: Delayed primary chest closure.


INDICATIONS: The patient is a newborn with diagnosis of hypoplastic left heart syndrome who 48 hours prior to the current procedure has undergone a modified stage 1 Norwood operation. Given the magnitude of the operation and the size of the patient (2.5 kg), we have elected to leave the chest open to facilitate postoperative management. He is now taken back to the operative room for delayed primary chest closure.


PREOP DX: Open chest status post modified stage 1 Norwood operation.


POSTOP DX: Open chest status post modified stage 1 Norwood operation.


ANESTHESIA: General endotracheal.


COMPLICATIONS: None.


FINDINGS: No evidence of intramediastinal purulence or hematoma. He tolerated the procedure well.


DETAILS OF PROCEDURE: The patient was brought to the operating room and placed on the operating table in the supine position. Following general endotracheal anesthesia, the chest was prepped and draped in the usual sterile fashion. The previously placed AlloDerm membrane was removed. Mediastinal cultures were obtained, and the mediastinum was then profusely irrigated and suctioned. Both cavities were also irrigated and suctioned. The drains were flushed and repositioned. Approximately 30 cubic centimeters of blood were drawn slowly from the right atrial line. The sternum was then smeared with a vancomycin paste. The proximal aspect of the 5 mm RV-PA conduit was marked with a small titanium clip at its inferior most aspect and with an additional one on its rightward inferior side. The sternum was then closed with stainless steel wires followed by closure of subcutaneous tissues with interrupted monofilament stitches. The skin was closed with interrupted nylon sutures and a sterile dressing was placed. The peritoneal dialysis catheter, atrial and ventricular pacing wires were removed. The patient was transferred to the pediatric intensive unit shortly thereafter in very stable condition. I was the surgical attending present in the operating room and in charge of the surgical procedure throughout the entire length of the case.''']
}
json_input_data = json.dumps(input_json_data)
with open(input_file_name, 'w') as f:
    f.write(json_input_data)

s3_client.put_object(
    Bucket=s3_bucket,
    Key="validation-input-json/{}".format(validation_file_name),
    Body=(bytes(json_input_data.encode("UTF-8"))),
)

In [14]:
from urllib.parse import urlparse

parsed_url = urlparse(transformer.output_path)
file_key = f"{parsed_url.path[1:]}/{validation_file_name}.out"
response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

data = json.loads(response["Body"].read().decode("utf-8"))
df = pd.DataFrame(data, columns=['summary'])
display(df)

with open(output_file_name, 'w') as f_out:
    json.dump(data, f_out, indent=4)

,summary
0,"A 23-year-old female with a history of allergies and asthma presents with worsening symptoms. She has tried Claritin and Zyrtec but they haven't worked well. She has tried Allegra and has tried over-the-counter sprays but no prescription nasal sprays. Her vital signs are stable, and she has mildly erythematous throat and nasal mucosa. She is diagnosed with allergic rhinitis and will try Zyrtec or loratadine. Samples of Nasonex were given for three weeks."
1,"A newborn with hypoplastic left heart syndrome underwent a modified stage 1 Norwood operation. The chest was left open to facilitate postoperative management, and the patient tolerated the procedure well. The sternum was closed with stainless steel wires, subcutaneous tissues with interrupted monofilament stitches, and the skin was closed with interrupted nylon sutures. The patient was transferred to the pediatric intensive unit in stable condition."


In [15]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-summarize-clinical-jsl-augmented-pip-2024-02-26-18-27-13-667


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

